In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10.0, 8.0) 
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'Spectral'

In [ ]:
def sigmoid(z) :
    """
    Returns the sigmoid function of the given input
    """
    return 1 /(1 + np.exp(-z))

def sigmoid_prime(z) :
    """
    Returns the derivative of the sigmoid function of the given input
    """
    return sigmoid(z)*(1-sigmoid(z))

def fp(x, eps):
    """
    Returns the value the smooth_relu takes in the interval -eps to eps
    
    The polynomial satifies the following conditions:
    fp(eps) = eps
    fp'(eps) = 1
    fp(-eps) = 0
    fp'(-eps) = 0
    
    """
    a_0 = eps/4
    a_1 = 0.5
    a_2 = 1/(4*eps)
    a_3 = 0
    
    return  a_0 + a_1*x + a_2*x**2 + a_3*x**3 

def smooth_relu(x, eps):
    """
    Returns a smoothed ReLU function with parameter epsilon
    """
    return fp(x, eps)*(np.abs(x) < eps) + x*(x >= eps)


def fp_prime(x, eps):
    """
    Returns derivative of the fp(x) with respect to x
    """
    a_0 = eps/4
    a_1 = 0.5
    a_2 = 1/(4*eps)
    a_3 = 0
    
    return  a_1 + 2*a_2*x + 3*a_3*x**2 

def smooth_relu_prime(x,eps):
    """
    Returns the derivative of the smooth relu function
    """
    return fp_prime(x, eps)*(np.abs(x) < eps) + 1*(x >= eps)

def relu(z): 
    """
    Returns the rectified linear unit applied to the given input 
    """
    return np.maximum(0,z)

def relu_prime(z) :
    """
    Returns the derivative of rectified linear unit applied to the given input 
    """
    return 1*(z>=0)

def smooth_relu_prime_wrt_eps(x,eps):
    """
    Returns the derivative of the smooth relu function
    with respect to epsilon given the input and parameter eps
    """
    return (1/4-(x**2)/(4*eps**2))*(np.abs(x) < eps)

In [ ]:
from sklearn.datasets import make_blobs

N = 400 # no. points per class

X, y = make_blobs(n_samples = N, centers = [(2,-2),(-2,-2),(-2,2),(2,2)], 
                  n_features = 2, cluster_std = 0.8, random_state = 1)
labels_orig = y
y = np.mod(labels_orig, 2)
y = y.reshape(y.shape[0],1)

plt.title('4 Gaussian Blobs dataset')
plt.plot(X[y[:,0]==0,0], X[y[:,0]==0,1], color='tab:red', marker='.', linestyle='None',label='class 1')
plt.plot(X[y[:,0]==1,0], X[y[:,0]==1,1], color='tab:blue', marker='.', linestyle='None', label='class 2')
plt.legend()
plt.show()


In [ ]:
def plotboundary_accuracy(X, y, W1, b1, W2, b2,eps):
    """
    Returns the accuracy for the given parameters and plots the decision boundary of the data
    """
    predicted_class = sigmoid(np.dot(smooth_relu(np.dot(X, W1) + b1,eps), W2) + b2)
    print('Accuracy: %.2f %%' % (100*np.mean((predicted_class >0.5) == y)))

    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),np.arange(y_min, y_max, 0.02))
    Z_axis = sigmoid(np.dot(smooth_relu(np.dot(np.c_[xx.ravel(), yy.ravel()], W1) + b1,eps), W2) + b2)
    Z_axis = (Z_axis > 0.5)
    Z_axis = Z_axis.reshape(xx.shape)
    plt.contourf(xx, yy, Z_axis, cmap=plt.cm.Spectral, alpha=0.8)
    plt.scatter(X[:, 0], X[:, 1], c=y[:,0], s=40, cmap=plt.cm.Spectral)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())

In [ ]:
def initialize_parameters(n_x,n_y,n_h):
    """
     Initialization of the weights and biases for the neural network
     n_x: number of inputs nodes
     n_y: number of output nodes
     n_h: number of hidden nodes
    """
    #np.random.seed(1)                   # Random seed for weights
    W1 = 1.0 * np.random.randn(n_x,n_h)/np.sqrt(n_x)  # [n_x, n_h ]
    b1 = np.zeros((1,n_h))              # [1, n_h ]
    W2 = 1.0 * np.random.randn(n_h,n_y)/np.sqrt(n_h) # [n_h, n_y ]
    b2 = np.zeros((1,n_y))              # [1, n_y ]
    #np.random.seed(None)              # Random seed for weighs
    
    return W1, b1, W2, b2   # Return initalise weights and biases

In [ ]:
def forward_prop(X, W1, b1, W2, b2,eps):
    """
     Return the parameters after one foward pass 
     N: Number of points per class
     X: dimension [2N, n_x]
     n_x: number of inputs nodes
     n_y: number of output nodes
     n_h: number of hidden nodes
     
    """
    Z1 = np.dot(X, W1) + b1   # [2N, n_h]
    A1 = smooth_relu(Z1,eps) # [2N, n_h]
    Z2 = np.dot(A1, W2) + b2  # [2N, n_y]
    A2 = sigmoid(Z2)          # [2N, n_y]
    
    return Z1, A1, Z2, A2

In [ ]:
def cost(Y, A2):
    """
    Returns the L2 error between the predicted values 
    by the network A2 and the actual values of the data Y
    """
    m = Y.shape[0]
    return np.sum(np.square(A2-Y))/m

In [ ]:
def back_prop(X,y, Z1, A1, Z2, A2, reg, eps):
    """
     Return the gradient of parameters with respect to the loss after one foward pass 
     reg: regularisation parameter
     eps: smooth relu parameter epsilon
     N: Number of points per class
     X: dimension [2N, n_x]
     n_x: number of inputs nodes
     n_y: number of output nodes
     n_h: number of hidden nodes
     """
    
    m = X.shape[0]                # Number of data points in total, 2N
    
    dZ2 = 2*(A2 - y)/m              # Derivative of cross-entropy loss is (A2-y), [m,1]
    dZ2 = dZ2*sigmoid_prime(Z2)   # Derivative with respect to Z2, [m,1]

    dW2 = np.dot(A1.T, dZ2)                     # Derivative with respect to W2, [n_h, n_y ]
    db2 = np.sum(dZ2, axis=0, keepdims=True)    # Derivative with respect to b2, [1, n_y ]

    dA1 = np.dot(dZ2, W2.T)                             # Derivative with respect to A1, [m, n_h]
    dZ1 = np.multiply(dA1, smooth_relu_prime(Z1,eps))  # Derivative with respect to Z1, [m, n_h]
    
    dW1 = np.dot(X.T, dZ1)                   # Derivative with respect to W1, [n_x, n_h]
    db1 = np.sum(dZ1, axis=0, keepdims=True) # Derivative with respect to b1, [1, n_h]

    dW2 = dW2 + reg * W2 # regularisation of weight W2
    dW1 = dW1 + reg * W1 # regularisation of weight W1

    return dW1, db1, dW2, db2, 

In [ ]:
def update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    """
    Returns the updated values of the parameters 
    after the given interation 
    """
    
    W1 = W1 - dW1 * learning_rate
    b1 = b1 - db1 * learning_rate
    W2 = W2 - dW2 * learning_rate
    b2 = b2 - db2 * learning_rate
    
    return W1, b1, W2, b2

## Separating dataset

In [ ]:
np.random.seed(5)
C = X.shape[0]
# You need to shuffle the data in order to get blue and red points in all the sets
perm = np.random.permutation(len(X)) # This allows us to shuffle data without losing the labels on the y-vector
X_shuffle= X[perm]
y_shuffle = y[perm]
np.random.seed(None)

# We define a size for train set
train_index = int(0.7*C)

In [ ]:
# Create sets, same method as for SGD
X_train, y_train = X_shuffle[:train_index,:], y_shuffle[:train_index] 
X_test, y_test = X_shuffle[train_index:,:], y_shuffle[train_index:]

In [ ]:
# Setting parameters
epochs = 1000
learning_rate = 0.1
runs = 100
reg = 1e-3
sample_size = 0.1
batch_size = int(sample_size*C*0.7)

In [ ]:
epsilon = np.around(np.arange(0.1,0.92,0.04),3)
E = len(epsilon)
accuracy_eps_test= np.zeros(E)  
accuracy_std_test= np.zeros(E)  
means_test = np.zeros(E)
std_test = np.zeros(E)

## SGD Training

In [ ]:
for k, eps in enumerate(epsilon):
    
    print(eps)
    accuracy_test = np.zeros(runs)
    losses_test =  np.zeros(runs)
    
    for n in range(runs):
        n_h = 3
        n_x = 2
        n_y = 1
        
        W1, b1, W2, b2 = initialize_parameters(n_x,n_y,n_h)
        
        for i in range(epochs):
            index = np.random.randint(0, train_index , batch_size)
            X_i = X_train[index,:]
            y_i = y_train[index]
    
            Z1, A1, Z2, A2 = forward_prop(X_i, W1, b1, W2, b2,eps)
        
            dW1, db1, dW2, db2 = back_prop(X_i, y_i, Z1, A1, Z2, A2, reg,eps)
            W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)
        
        pc_test =sigmoid(np.dot(smooth_relu(np.dot(X_test, W1) + b1,eps), W2) + b2)
        accuracy_test[n] = (100*np.mean((pc_test >0.5) == y_test))
    
        reg_loss = 0.5*reg*np.sum(W1*W1) + 0.5*reg*np.sum(W2*W2)
        losses_test[n] =  cost(y_test, pc_test) + reg_loss  

    accuracy_eps_test[k] = np.mean(accuracy_test)
    accuracy_std_test[k] = np.std(accuracy_test)
    means_test[k], std_test[k]= np.mean(losses_test), np.std(losses_test) 

In [ ]:
for i in range(E): 
    print('epsilon:',round(epsilon[i],3), 'Accuracy: %.5f %%' %accuracy_eps_test[i], 'Std: %.5f %%' %accuracy_std_test[i],  'Loss: %.4f' %means_test[i],   'Std %.4f' %std_test[i])

In [ ]:
indices = means_test.argsort()[:3][::-1]
epsilon_test = np.array(epsilon)[indices.astype(int)] 
for i in indices: 
     print('epsilon:',round(epsilon[i],3), 'Accuracy: %.5f %%' %accuracy_eps_test[i], 'Std: %.5f %%' %accuracy_std_test[i],  'Loss: %.4f' %means_test[i],   'Std %.4f' %std_test[i])

In [ ]:
# Plotting
fig,ax2 = plt.subplots(figsize=(16, 8))
ax2.set_xlabel(r'$\varepsilon$',  fontsize=20)
ax2.set_ylabel('Loss', color='tab:orange', fontsize=20)
ax2.errorbar(epsilon, means_test, linestyle='None', marker='o', color='tab:orange', markersize=8)
ax2.tick_params(axis='y', labelcolor='tab:orange')
plt.title(r' Loss with respect to $\varepsilon$ values 4-blobs',fontsize=20)
plt.xticks(fontsize = 15) 
plt.yticks(fontsize = 15)

ax3 = ax2.twinx()  # instantiate a second axes that shares the same x-axis
ax3.set_ylabel('Accuracy', color='tab:green', fontsize=20)  # we already handled the x-label with ax1
ax3.errorbar(epsilon, accuracy_eps_test, linestyle='None', marker='o', color='tab:green')
ax3.tick_params(axis='y', labelcolor='tab:green')

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()